In [4]:
# Imports

import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scipy.ndimage import gaussian_filter1d
from scipy.signal import medfilt
from sklearn.model_selection import train_test_split

In [5]:
# Load Data

data = pd.read_csv("data/weatherHistory.csv")
temp_data = data.iloc[:,3:4].values #temperature column only
temp_data

array([[ 9.47222222],
       [ 9.35555556],
       [ 9.37777778],
       ...,
       [22.03888889],
       [21.52222222],
       [20.43888889]])

In [6]:
# Optional Filter

filter_active = 0
if filter_active == True:
  temp_data = medfilt(temp_data, 3)
  temp_data = gaussian_filter1d(temp_data, 1.2)

In [7]:
# Encode the data

mms = MinMaxScaler(feature_range = (0,1))
temp_data = mms.fit_transform(temp_data)

In [8]:
# Train data (OPTION 1)

x_train = []
y_train = []
n_future = 7     #7 days temperature forecast
n_past = 30      #Past 30 days 

for i in range(0,len(temp_data)-n_past-n_future+1):
    x_train.append(temp_data[i : i + n_past , 0])     
    y_train.append(temp_data[i + n_past : i + n_past + n_future , 0 ])

x_train , y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0] , x_train.shape[1], 1) )


# ----------------------------------------------------------------------------

# Train Data (OPTION 2)

# x_train, y_train = train_test_split(temp_data, 
#                                     test_size=0.25, 
#                                     random_state=0)

In [9]:
# LSTM Model Defninition

model = Sequential()
model.add(Bidirectional(LSTM(units=50, 
                             activation= "sigmoid", 
                             return_sequences=True, 
                             input_shape = (x_train.shape[1], 1))))
model.add(Dropout(0.2))
model.add(LSTM(units= 50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units= 50))
model.add(Dense(units = n_future , activation="sigmoid"))  

In [10]:
# Train Model

model.compile(optimizer='adam', 
              loss = 'mean_squared_error', 
              metrics='accuracy')
model.fit(x_train, 
          y_train, 
          epochs=100, 
          batch_size=128)

Epoch 1/100
754/754 [==============================] - 136s 180ms/step - loss: 0.0110 - accuracy: 0.2074
Epoch 2/100
754/754 [==============================] - 141s 187ms/step - loss: 0.0054 - accuracy: 0.4297
Epoch 3/100
177/754 [======>.......................] - ETA: 2:08 - loss: 0.0044 - accuracy: 0.4662

KeyboardInterrupt: 